In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [47]:
weather_df = pd.read_csv('../WeatherPy/WeatherPyCities.csv')
weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Humidity,Temp,Cloudiness,Wind Speed,Country
0,0,saldanha,-33.0117,17.9442,71.0,57.25,0,10.20,AU
1,1,sigli,5.3848,95.9609,70.0,84.60,57,1.74,MU
2,2,haines junction,60.7522,-137.5108,51.0,35.55,100,5.75,AU
3,3,manjacaze,-24.7117,33.8828,96.0,69.53,98,5.14,SA
4,4,sri aman,1.2376,111.4621,74.0,86.05,20,2.30,AR


In [48]:
# Store latitudes & longitudes in locations
lat_lon = weather_df[['Latitude', 'Longitude']]

# Fill NaN values and convert to float
humid = weather_df["Humidity"]
lat_lon.head()

,Latitude,Longitude
0,-33.0117,17.9442
1,5.3848,95.9609
2,60.7522,-137.5108
3,-24.7117,33.8828
4,1.2376,111.4621


In [55]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create heat layer
heat_layer = gmaps.heatmap_layer(lat_lon, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [58]:
# Create criteria for the vacation 
# A max temperature < 80 degrees but > 70; Wind speed < 10 mph; Zero cloudiness 
crit_temperature = (weather_df['Temp'] < 80) & (weather_df.Temp > 70)
crit_windSpeed = (weather_df['Wind Speed'] < 10)
crit_cloudiness = (weather_df['Cloudiness'] == 0)
final_criteria = crit_temperature & crit_windSpeed & crit_cloudiness 

# Filter the weather_df dataframe
ideal_weather_df = weather_df[final_criteria]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head()

,index,Unnamed: 0,City,Latitude,Longitude,Humidity,Temp,Cloudiness,Wind Speed,Country
0,88,96,atar,20.5169,-13.0499,15.0,75.97,0,3.80,TO
1,108,118,marsh harbour,26.5412,-77.0636,77.0,75.85,0,9.26,MU
2,109,119,puerto penasco,31.3167,-113.5333,82.0,70.92,0,3.00,RU
3,205,227,saint-pierre,-21.3393,55.4781,80.0,72.05,0,4.61,EG
4,224,248,sanford,28.8006,-81.2731,86.0,72.68,0,5.75,MG


In [60]:
hotel_df = ideal_weather_df
# Add column for Hotel Name 
hotel_df['Hotel Name'] = ""
hotel_df.head()

,index,Unnamed: 0,City,Latitude,Longitude,Humidity,Temp,Cloudiness,Wind Speed,Country,Hotel Name
0,88,96,atar,20.5169,-13.0499,15.0,75.97,0,3.80,TO,
1,108,118,marsh harbour,26.5412,-77.0636,77.0,75.85,0,9.26,MU,
2,109,119,puerto penasco,31.3167,-113.5333,82.0,70.92,0,3.00,RU,
3,205,227,saint-pierre,-21.3393,55.4781,80.0,72.05,0,4.61,EG,
4,224,248,sanford,28.8006,-81.2731,86.0,72.68,0,5.75,MG,


In [62]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Use try-except to skip missing data points
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field: skipping.")
#hotel_df.info()

Missing field: skipping.
Missing field: skipping.


In [63]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [64]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…